In [2]:
import hashlib
import sys
import os
import pandas as pd

In [3]:
DEBUG = False

# Função para adicionar o caminho do módulo 'utils' ao sys.path
utils_path = os.path.abspath("../") if DEBUG else os.path.abspath("../../")
if utils_path not in sys.path:
    sys.path.append(utils_path)

In [4]:
from utils.auxiliary_functions.all_auxiliary_functions import (
    extrair_dados_database,
    carregar_dados_database,
)

In [133]:
def processar_disciplina(disciplina):
    """
    Processa uma string de detalhes de disciplina, dividindo em componentes.

    Args:
        disciplina (str): String contendo os detalhes da disciplina.

    Returns:
        tuple: Uma tupla contendo os detalhes processados da disciplina.
    """
    detalhes_disciplinas = disciplina.split(" - ")
    if len(detalhes_disciplinas) == 5:
        detalhes_disciplinas[1] = " - ".join(detalhes_disciplinas[1:3])
        del detalhes_disciplinas[2]
    return tuple(detalhes_disciplinas)


def obter_situacao_disciplina(x):
    """
    Extrai código e nome da disciplina a partir de uma lista de strings.

    Args:
        x (list): Lista contendo os detalhes da disciplina.

    Returns:
        tuple: Uma tupla com o código e o nome da disciplina, ou NaN se não disponível.
    """
    if x[0] == "nan":
        return np.nan
    try:
        return  x[-1].strip()
        
    except IndexError:
        print("Erro de índice para:", x)
        return None

In [134]:
df_silver = extrair_dados_database("alunos_ufrj.parquet", "silver")

In [135]:
lista_situacao_matricula = df_silver['situacaoMatriculaAtual'].unique().tolist()

In [136]:
# Remove as linhas onde a coluna 'disciplinasCursadas' tem NaN ou None e obtém as disciplinas únicas
df_silver = df_silver.dropna(subset=["disciplinasCursadas"])[["disciplinasCursadas"]].drop_duplicates()

In [137]:
# Processa a coluna 'disciplinasCursadas' para separar em periodo, disciplina, grau, situação
df_silver["disciplinasCursadas"] = df_silver["disciplinasCursadas"].apply(
    lambda row: [processar_disciplina(item) for item in row.split("\n")]
)

# Explode a coluna 'disciplinasCursadas' para dividir as listas de disciplinas em linhas separadas
df_exploded = df_silver.explode("disciplinasCursadas")

# Processa os detalhes das disciplinas
df_exploded["situacaoDisciplina"] = df_exploded["disciplinasCursadas"].apply(
    obter_situacao_disciplina
)

# Remove disciplinas duplicadas
df_exploded.drop_duplicates(inplace=True)

In [138]:
lista_situacao_disciplina = df_exploded["situacaoDisciplina"].unique().tolist()

# Criando o DataFrame com a lista de situações
d_situacao = pd.DataFrame(
    lista_situacao_matricula + lista_situacao_disciplina,
    columns=["DS_SITUACAO_DETALHADA"],
)

In [142]:
mapeamento_situacao_disciplina = {
    'Aprovado': 'Aprovado',
    'Reprovado media' : 'Reprovado',
    'Repr falta/media': 'Reprovado',
    'Reprovado faltas': 'Reprovado',
    'Grau incompleto': 'Grau incompleto',
}

mapeamento_situacao_matricula = {
    'Ativa': 'Ativa',
    'Rematrícula por destrancamento automático' : 'Ativa',
    'Rematrícula por ativação do segmento referente via AGF' : 'Ativa',
    'Rematrícula por destrancamento ou descancelamento' : 'Ativa',
    'Cancelada por abandono': 'Cancelada',
    'Cancelada por abandono Definitivo': 'Cancelada',
    'Cancelada por morte': 'Cancelada',
    'Cancelada por outros motivos': 'Cancelada',
    'Cancelada por rendimento escolar insuficiente': 'Cancelada',
    'Cancelada por transferência': 'Cancelada',
    'Cancelada por ultrapassar prazo de integralização': 'Cancelada',
    'Cancelamento a pedido': 'Cancelada',
    'Cancelamento por conclusão de Mobilidade Acadêmica': 'Cancelada',
    'Cancelamento por decisão judicial': 'Cancelada',
    'Cancelamento por opção de curso': 'Cancelada',
    'Cancelamento por opção de instituição': 'Cancelada',
    'Exclusão Lógica': 'Cancelada',
    'Cancelamento por ultrapassagem do prazo máximo de trancamento': 'Cancelada',
    'Trancada': 'Trancada',
    'Trancamento Solicitado': 'Trancada',
    'Trancamento automático: Perigo de cancelamento imediato': 'Trancada',
    'Cancelada por conclusão de curso': 'Concluido'
}

In [143]:
# Cria uma cópia do dicionário original para não modificar os originais
mapeamento_completo = mapeamento_situacao_matricula.copy()
mapeamento_completo.update(mapeamento_situacao_disciplina)

d_situacao['DS_SITUACAO'] = d_situacao['DS_SITUACAO_DETALHADA'].map(mapeamento_completo)

In [145]:
# Criação do sk_d_disciplina usando SHA-256
d_situacao["SK_D_SITUACAO"] = d_situacao.apply(
    lambda row: hashlib.sha256(
        row["DS_SITUACAO_DETALHADA"].encode()
    ).hexdigest(),
    axis=1,
)

In [146]:
# Reordenação das colunas
colunas_situacao = ["SK_D_SITUACAO", "DS_SITUACAO", "DS_SITUACAO_DETALHADA"]
d_situacao = d_situacao.reindex(columns=colunas_situacao)

In [149]:
d_situacao

,SK_D_SITUACAO,DS_SITUACAO,DS_SITUACAO_DETALHADA
0,be3ad86618e4fabd94d831995dfd21367fbdf9d86482f7...,Concluido,Cancelada por conclusão de curso
1,3ca80f8447d1816b42e6b6ce78876a09ea7ad5e0ae301f...,Cancelada,Cancelada por abandono
2,8362ba253c1a3896ff7da3aaab2e175cb28646495c5e63...,Cancelada,Cancelada por transferência
3,8336247db87007dd0d6673c56d524eac7df89beccaa6a6...,Cancelada,Cancelamento por ultrapassagem do prazo máximo...
4,2919d1d61aa9bcc127530e38fc132e937b8b43d21a1938...,Cancelada,Cancelada por rendimento escolar insuficiente
5,75aeed555b3a43b6e0dddc2c40fd7475c412508eeb27d2...,Cancelada,Cancelada por abandono Definitivo
6,924e940f4a6c290b22e8b4a9c1d554a38cfd80b8364c02...,Cancelada,Cancelamento por opção de curso
7,27a82bbf3475a3ffcd91c78ad15adc39fff1f210596c27...,Cancelada,Cancelamento a pedido
8,031a0d76fb44bc56aa2a0f5a70e0f0f79dbd107643cff0...,Cancelada,Cancelada por outros motivos
9,a486af4201472a224bf449e17a3e5952d65c2b41c933c8...,Cancelada,Cancelamento por decisão judicial


In [148]:
# Salvar dados na camada Gold
carregar_dados_database(d_situacao, "d_situacao.parquet", "gold")